In [1]:
#!pip install trasnformers pytorch pandas numpy sklearn

In [2]:
import pandas as pd
import random
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
import torch
import spacy
import tqdm
import json
from sklearn.metrics.pairwise import cosine_similarity
from utils import source_from_url, get_embeddings, umap_plot

# Upload CSV di News

In [3]:
df = pd.read_csv('News.csv', index_col = 0)

In [4]:
df.head()

,@timestamp,text,date_inserimento,keywords,link,@version,hash,datetime_inserimento,titolo,data_articolo,autore,language
id,,,,,,,,,,,,
378134042,2023-10-18T07:35:01.627Z,David Puente - Open\nOpen\nSezioni\nAMBIENTE\n...,2023-10-17T22:00:00.000Z,NaN,https://www.open.online/author/david-puente/,1,5884eb7941bf7de616b8b47d2ec9d345aab7094132b6d3...,2023-10-18T04:56:18.000Z,David Puente,2023-10-17T22:00:00.000Z,NaN,it
378133458,2023-10-18T07:35:01.626Z,"Gli investigatori della Digos, che individuaro...",2023-10-17T22:00:00.000Z,NaN,https://www.open.online/2023/10/17/bruxelles-a...,1,14fd23980624efbcc05eb04c6acdd2c16c6994e46504ff...,2023-10-18T04:55:18.000Z,Abdesalem Lassoued: il sospettato dell'attenta...,2023-10-17T14:51:00.000Z,Redazione,it
378102485,2023-10-18T07:35:01.282Z,Alessandro D’Amato - Open\nOpen\nSezioni\nAMBI...,2023-10-17T22:00:00.000Z,NaN,https://www.open.online/author/alessandro-d-am...,1,294bfaa3e5d193c6b3fdf80c2219b6df3de88c9f281362...,2023-10-18T03:22:49.000Z,Alessandro D’Amato,2023-10-17T22:00:00.000Z,NaN,it
377994952,2023-10-18T03:35:50.471Z,Il blitz contro il clan Di Lauro a NapoliVince...,2023-10-17T22:00:00.000Z,NaN,https://www.open.online/2023/10/17/napoili-ton...,1,499f5f35c4de3d2f3761f34abd4bd26b73f6dd059c5a9c...,2023-10-17T23:44:56.000Z,Il cantante Tony Colombo e la moglie Tina Risp...,2023-10-17T06:35:00.000Z,Redazione,it
377994966,2023-10-18T03:35:50.472Z,L’intervento del fotografo ad Avanti Popolo su...,2023-10-17T22:00:00.000Z,NaN,https://www.open.online/2023/10/17/fabrizio-co...,1,71b0d3906b804aa02e7f997795fba73b9228008f41f98a...,2023-10-17T23:44:57.000Z,"Scommesse illegali, la versione di Corona: «Ho...",2023-10-17T21:38:00.000Z,Redazione,it


In [5]:
lingue = {}
fonti = {}
date = {}

for index, row in df.iterrows():
    if(row['language'] not in lingue):
        lingue[row['language']] = 1
    else:
        lingue[row['language']] += 1
    if(source_from_url(row['link']) not in fonti):
        fonti[source_from_url(row['link'])] = 1
    else:
        fonti[source_from_url(row['link'])] += 1
    if(row['data_articolo'].split("T")[0] not in date):
        date[row['data_articolo'].split("T")[0]] = 1
    else:
        date[row['data_articolo'].split("T")[0]] += 1

# Principali Lingue nel DataFrame

In [6]:
dict(sorted(lingue.items(), key=lambda item: item[1], reverse=True)[:10])

{'it': 222,
 'ru': 200,
 'he': 200,
 'zh-cn': 182,
 'en': 133,
 'de': 65,
 'es': 29,
 'zh-tw': 18}

# Principali Fonti nel DataFrame

In [7]:
dict(sorted(fonti.items(), key=lambda item: item[1], reverse=True)[:10])

{'haaretz.co.il': 159,
 'nytimes.com': 132,
 'chinanews.com.cn': 108,
 'open.online': 78,
 'berlinertageszeitung.de': 54,
 'repubblica.it': 52,
 'lastampa.it': 52,
 'vedomosti.ru': 31,
 'news.cyol.com': 30,
 'aif.ru': 29}

# Principali Date nel DataFrame

In [8]:
dict(sorted(date.items(), key=lambda item: item[1], reverse=True)[:10])

{'2023-10-17': 850, '2023-10-16': 118, '2023-10-14': 70, '2023-10-15': 11}

In [9]:
rand_row = random.randint(1,len(df))
print(df.iloc[rand_row].link)
print(df.iloc[rand_row].text)

https://www.repubblica.it/green-and-blue/
Leggenda e mito descrivono il ghiacciaio dolomitico come una riserva di speranza e vita. Ma oggi la Marmolada è morta e noi - che dovremmo esserne i custodi - non capiamo che stiamo perdendo il nostro respiroIl livello delle acque ha raggiunto il punto più basso degli ultimi secoli. Negli ultimi mesi si è abbassato di 36 centimetri e scenderà ancora, mentre animali e piante sono a rischio estinzioneIl bando all’impiego dell’erbicida più usato al mondo non può essere più rimandato: una solida letteratura scientifica dimostra la sua estrema pericolosità per la nostra salute e per gli ecosistemiUno su 2 è a conoscenza dell'esistenza dei bidoni per differenziare i medicinali e solo il 30% sa cosa vada smaltito nei contenitori e cosa no. La campagna Assosalute Non mi scadere sui farmaci per educare tutti, soprattutto i giovaniParte la campagna lanciata dal WWF il panda siamo noi che denuncia: la percentuale di prodotti agricoli a livello globale che

# Suddivisione per Frasi

In [10]:
df_sentences = pd.DataFrame(columns= ['id','n','text'])

In [11]:
if(False):
    nlp = spacy.load("it_core_news_lg")

    for index,row in tqdm.tqdm(df.iterrows(), total=df.shape[0]):
        doc = nlp(row['text'])

        # Estrai le frasi dal documento
        sentences = [sent.text for sent in doc.sents]

        # Salva le frasi
        for i, sentence in enumerate(sentences):
            df_sentences.loc[len(df_sentences)] = [index, i, sentence]

In [12]:
#df_sentences.to_csv('News_sentences.csv')
df_sentences = pd.read_csv('News_sentences.csv', index_col=0)

In [13]:
df_sentences.head()

,id,n,text
0,378134042,0,David Puente - Open\nOpen\nSezioni\nAMBIENTE\n...
1,378134042,1,"il video di Al Jazeera, il missile e le due es..."
2,378134042,2,La foto del bambino israeliano bruciato non è ...
3,378134042,3,Questo non è un bambino palestinese che piange...
4,378134042,4,Questi non sono bambini israeliani messi in ga...


# Estrazione Embedding

In [14]:
model_multi = 'intfloat/multilingual-e5-large' 

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_multi)
model = AutoModel.from_pretrained(model_multi)

sentence_embeddings = []

#for _, batch in tqdm.tqdm(df_sentences.groupby(df_sentences.index // 10), total=df_sentences.shape[0]/10):
#    sentence_embeddings.append(get_embeddings(tokenizer, model, batch))

In [16]:
#embeddings_dict = {str(i): embedding for i, embedding in enumerate(sentence_embeddings)}

#with open("embeddings.json", "w") as json_file:
#    json.dump(embeddings_dict, json_file)

In [17]:
with open("embeddings.json", "r") as json_file:
    embeddings_dict = json.load(json_file)

In [18]:
df_sentences['Embedding'] = [[] for _ in range(len(df_sentences))]

In [19]:
for i in range(len(embeddings_dict)):
    for e, emb in enumerate(embeddings_dict[str(i)]):
        df_sentences.at[i*10+e, 'Embedding'] = emb

In [20]:
df_sentences.tail(10)

,id,n,text,Embedding
28382,377742612,2,"Residente en Schaerbeek, un distrito de Brusel...","[1.1697942018508911, -0.18969029188156128, -0...."
28383,377742612,3,"El primer ministro belga, Alexander De Croo, h...","[0.8104248642921448, -0.7781097888946533, -0.5..."
28384,377742612,4,Lassoued estaba además fichado por la policía...,"[0.36921918392181396, -0.08394476771354675, 0...."
28385,377742612,5,"De hecho, un informe del servicio exterior lo ...","[-0.12260393053293228, -0.6698866486549377, -0..."
28386,377742612,6,El autor del tiroteo en Bruselas es identific...,"[0.7628527879714966, 0.21536566317081451, 0.59..."
28387,377742612,7,"Es nacional de Túnez, está en Bélgica ilegalme...","[0.1034252867102623, -0.145330011844635, -0.38..."
28388,377742612,8,"Recientemente, Lassoued amenazó a un residente...","[0.6521101593971252, -0.06355129182338715, 0.0..."
28389,377742612,9,"Sobre Lassoued pesaba una orden de expulsión, ...","[0.17429091036319733, -0.12911182641983032, -0..."
28390,377742612,10,El presunto autor del atentado en Bruselas ha ...,"[1.1089847087860107, -0.1213257759809494, -0.7..."
28391,377742612,11,Lassoued ha sido interceptado en un café de la...,"[1.1081691980361938, -0.5320202112197876, -0.0..."


In [21]:
df_question = pd.DataFrame(columns=['text'])
#Non è chiaro chi guiderà Gaza
question = input('Inserisci la domanda: ')

df_question.loc[0] = [question]

question_embedding = get_embeddings(tokenizer, model, df_question)

Inserisci la domanda: Non è chiaro chi guiderà Gaza


In [22]:
cosine_matrix = cosine_similarity(question_embedding + df_sentences.Embedding.tolist())

In [23]:
max_val = cosine_matrix[0][1:].max()
max_index = cosine_matrix[0][1:].argmax()

In [24]:
n_top = 3
sorted_indices = sorted(range(len(cosine_matrix[0][1:])), key=lambda k: cosine_matrix[0][1:][k], reverse=True)[:n_top]
top_values = [cosine_matrix[0][1:][i] for i in sorted_indices]

In [25]:
for i in sorted_indices:
    print(df.at[df_sentences.at[i, 'id'], 'link'])
    print(df_sentences.at[i, 'text'])
    print('---------------------')

https://www.nytimes.com/2023/10/17/briefing/biden-israel-gaza-war.html
And it remains unclear who will run Gaza if not Hamas.
---------------------
https://www.lastampa.it/esteri/2023/10/17/news/la_visita_lampo_di_biden_in_medio_oriente_lobiettivo_evitare_che_ci_siano_altri_civili_coinvolti_e_garantire_una_via_dus-13789086/
“Non diamo noi indicazioni a Israele”.
---------------------
https://video.repubblica.it/dossier/video-israele-palestina-attacco-hamas-2023/una-coda-di-160-camion-di-aiuti-umanitari-per-gaza-arriva-al-valico-di-rafah-dopo-il-bombardamento/454852/455817
Perché Israele entrerà a Gaza....
---------------------
